In [1]:
# CARLA Model Testing Script with Prediction Logging

import os
import time
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import carla

In [2]:
# === Define Driving Model ===
class DrivingModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ReLU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ReLU(),
            nn.Conv2d(36, 64, 3),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(64*51*51, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        )

    def forward(self, x):
        return self.fc(self.cnn(x))


In [3]:
# === Setup Device and Model ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DrivingModel().to(device)
model.load_state_dict(torch.load('driving_model.pth', map_location=device))
model.eval()


DrivingModel(
  (cnn): Sequential(
    (0): Conv2d(3, 24, kernel_size=(5, 5), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(24, 36, kernel_size=(5, 5), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(36, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=166464, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=3, bias=True)
  )
)

In [4]:
# === Define Image Transform ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:

# === Connect to CARLA ===
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

In [6]:

# === Spawn Vehicle and Camera ===
blueprint_lib = world.get_blueprint_library()
vehicle_bp = blueprint_lib.filter('vehicle.*')[0]
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

camera_bp = blueprint_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '480')
camera = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=1.5, z=2.4)), attach_to=vehicle)


In [7]:
# === Define Image Processing Function ===
def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))[:, :, :3]
    img = Image.fromarray(array).convert('RGB')
    tensor = transform(img).unsqueeze(0).to(device)
    return tensor

In [8]:
# === Initialize Logging ===
all_preds = []
frame_count = 0
MAX_FRAMES = 500

In [9]:
# === Define Image Callback with Logging ===
def image_callback(image):
    global frame_count
    if frame_count >= MAX_FRAMES:
        camera.stop()
        return

    tensor = process_image(image)
    with torch.no_grad():
        controls = model(tensor).cpu().numpy()[0]

    all_preds.append(controls.tolist())
    frame_count += 1

    vehicle.apply_control(carla.VehicleControl(
        steer=float(np.clip(controls[0], -1, 1)),
        throttle=float(np.clip(controls[1], 0, 1)),
        brake=float(np.clip(controls[2], 0, 1))
    ))

In [10]:
# === Start Camera Stream ===
camera.listen(image_callback)

In [11]:
# === Wait for Completion ===
while frame_count < MAX_FRAMES:
    time.sleep(0.1)

In [12]:
# === Save Predictions ===
df = pd.DataFrame(all_preds, columns=['steer', 'throttle', 'brake'])
df.to_csv('test_predictions.csv', index=False)
print("Saved predictions to test_predictions.csv")


Saved predictions to test_predictions.csv


In [1]:
import os
print("Saving to:", os.getcwd())


Saving to: c:\Users\TANMAY\OneDrive\Desktop\Major project self driving\New
